In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext Cython

In [4]:
%%cython -a
cimport cython
from libc.stdlib cimport malloc, free
from libc.stdio cimport printf
from cython cimport sizeof, NULL
from libc.stdio cimport printf
import numpy as np
cimport numpy as np
from libc.math cimport sqrt


ctypedef np.float64_t dtype_t

    
##
"""First layer simplex. The Vertex"""
"""Below functions to create an instance of the struct"""
ctypedef struct Vertex:
    double x
    double y
    int vid_nr

    


cdef Vertex create_vertex(double x_coord, double y_coord, int vid_number) nogil:
    cdef Vertex v
    v.x = x_coord
    v.y = y_coord
    v.vid_nr = vid_number
    return v





##
"""Third layer simples. The Face"""
"""Functions to create instances of Face"""
ctypedef struct Face:
    Vertex ver1
    Vertex ver2
    Vertex ver3
    int fid_nr


    

cdef Face create_face(Vertex vertex1, Vertex vertex2, Vertex vertex3, int fid_number) nogil:
    cdef Face f
    f.ver1 = vertex1
    f.ver2 = vertex2
    f.ver3 = vertex3
    f.fid_nr = fid_number
    return f


##
"""The second layer simplex. The edge"""
"""Function to create an Instance of Edge struct"""
ctypedef struct Edge:
    Vertex v1
    Vertex v2
    int eid_nr


cdef Edge create_edge(Vertex vertex1, Vertex vertex2, int eid_number) nogil:
    cdef Edge e
    e.v1 = vertex1
    e.v2 = vertex2
    e.eid_nr = eid_number
    return e


"""return magnitude of vector"""
#works
cdef inline double magnitude(double x, double y) nogil:
    return sqrt(x*x+y*y)



"""return dot product with respect to origin"""
#works
cdef inline double dot_product(Vertex v1, Vertex v2) nogil:
    return v1.x * v2.x + v1.y * v2.y



"""Distance between vertices"""
#works

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double distance(Vertex v1, Vertex v2) nogil:
    return sqrt((v1.x - v2.x)**2 + (v1.y - v2.y)**2)



"""Circumcenter"""
#works

#input 3 vertexes to calculate circumcenter of circumcircle
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline Vertex circumcenter(Vertex v1, Vertex v2, Vertex v3) nogil:
    cdef Vertex center
    #inverse of determinant method (by wedge product)
    cdef double D = (v1.x*(v2.y-v3.y) + v2.x*(v3.y-v1.y) + v3.x*(v1.y-v2.y))*2
    center.x = (1/D)*((v1.x**2 + v1.y**2)*(v2.y-v3.y) + (v2.x**2 + v2.y**2)*(v3.y-v1.y) + (v3.x**2 + v3.y**2)*(v1.y-v2.y))
    center.y = (1/D)*((v1.x**2 + v1.y**2)*(v3.x-v2.x) + (v2.x**2 + v2.y**2)*(v1.x-v3.x) + (v3.x**2 + v3.y**2)*(v2.x-v1.x))
    
    return center


"""Circumradius"""
#works

#you have to be retarded to not know what this returns
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double circumradius(Vertex v1, Vertex v2, Vertex v3) nogil:
    cdef Vertex ccenter = circumcenter(v1, v2, v3)
    cdef double radius = sqrt((v1.x-ccenter.x)**2 + (v1.y-ccenter.y)**2)
    
    return radius



"""Check if Vertex p is in Circumcircle of v1v2v3"""
#works

#returns 0 or 1 based on condition if p is in C(v)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int in_circumcircle(Face f, Vertex p) nogil:
    cdef Vertex c_center = circumcenter(f.ver1, f.ver2, f.ver3)
    cdef double c_radius = circumradius(f.ver1, f.ver2, f.ver3)
    cdef double distance_from_circumcenter = distance(c_center, p)
    return distance_from_circumcenter <= c_radius




"""Create the super Triangle"""
"""First create the max rectangle"""
"""Then set the triangle with sqrt3 formulas"""
#works

#return a pointer to a [3] array with 3 Vertex objects
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face create_super_triangle(Vertex* points, int num_points) nogil:
    cdef Face super_triangle
    
    cdef double min_x = points[0].x
    cdef double min_y = points[0].y
    cdef double max_x = points[3].x
    cdef double max_y = points[3].y

    cdef int i

    for i in range(num_points):
        if points[i].x < min_x:
            min_x = points[i].x

        if points[i].y < min_y:
            min_y = points[i].y
            
    for i in range(num_points):
        if points[i].x > max_x:
            max_x = points[i].x

        if points[i].y > max_y:
            max_y = points[i].y
            
    cdef double a = (max_x - min_x)
    cdef double b = (max_y - min_y)

    

    super_triangle.ver1.x = min_x + a*0.5
    super_triangle.ver1.y = max_y + a*0.5
    super_triangle.ver1.vid_nr = 1


    super_triangle.ver2.x = max_x + b
    super_triangle.ver2.y = min_y - 1
    super_triangle.ver2.vid_nr = 2


    super_triangle.ver3.x = min_x - b
    super_triangle.ver3.y = min_y - 1
    super_triangle.ver3.vid_nr = 3

    
    super_triangle.fid_nr = 1


    return super_triangle






"""A function to allocate the Vertex struct in an array for testing purposes"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Vertex* alloc_varr(int size, double var) nogil:
    cdef Vertex* varray = <Vertex*>malloc(size * sizeof(Vertex))
    cdef size_t i
    for i in range(size):
        varray[i] = create_vertex(var, var, i)
    return varray





"""A function to allocate the Edge struct in an array for testing purposes"""
#logic mistake. do not try yet
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Edge* alloc_earr(Vertex* vertex_array, int varr_size) nogil:
    if varr_size%2!=0 or varr_size==0:
        printf("!number of vertices is not even!")
        return NULL
    cdef int earr_size = varr_size//2
    cdef Edge* earray = <Edge*>malloc(earr_size * sizeof(Edge))
    cdef size_t i
    for i in range(earr_size):
        earray[i] = create_edge(vertex_array[i*2], vertex_array[i*2+1], i)
    return earray





"""A function like the one below to realloc a vertex array instead of a face array"""

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Vertex* realloc_to_varray(Vertex* old_array, int current_size, Vertex new_vertex) nogil:
    cdef int new_size = current_size + 1
    cdef Vertex* new_array = <Vertex*>malloc(new_size * sizeof(Vertex))
    cdef size_t i
    for i in range(current_size):
        new_array[i] = old_array[i]
    new_array[current_size] = new_vertex
    free(old_array)
    return new_array



"""A function that takes a pointer to an array of Face structs. It creates a new array 1 size larger."""
"""It then appends the -new face- to it. Returns the new array. Frees the old one from memory"""
"""ATTENTION!!::: THE FACE* MESH INPUT MUST BE A MALLOC'ED ARRAY OR A POINTER TO A STATICALLY DECLARED ONE BY REFERENCING"""
#works
@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline Face* realloc_to_mesh(Face* mesh, int current_size, Face new_face) nogil:
    cdef int new_size = current_size + 1
    cdef Face* new_mesh = <Face*>malloc(new_size * sizeof(Face))
    cdef size_t i
    for i in range(current_size):
        new_mesh[i] = mesh[i]
    new_mesh[current_size] = new_face
    free(mesh)
    return new_mesh





"""The most important part to have valid faces with no crossings. It takes a list of free vertexes and oriens them c-clockwise"""
"""Uses the Dot product orthogonality and Bubble Sort"""
#SUPER EXPERIMENTAL
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void sort_by_angle_clockwise(Vertex* free_vertexes, Vertex center_vertex, int size) nogil:
    cdef int vertexes_over_x_axis = 0
    cdef int vertexes_under_x_axis = 0
    """DON'T FORGET TO FREE"""
    ###
    cdef Vertex* displaced_vertexes_overx = <Vertex*>malloc(sizeof(Vertex))
    cdef Vertex* displaced_vertexes_underx = <Vertex*>malloc(sizeof(Vertex))
    ###
    

    cdef double x_newcoord
    cdef double y_newcoord
    
    cdef size_t i
    
    for i in range(size):
        x_newcoord = free_vertexes[i].x - center_vertex.x
        y_newcoord = free_vertexes[i].y - center_vertex.y

        if y_newcoord >= 0:
            vertexes_over_x_axis = vertexes_over_x_axis + 1
            displaced_vertexes_overx = realloc_to_varray(displaced_vertexes_overx, vertexes_over_x_axis-1, free_vertexes[i])
            
        elif y_newcoord < 0:
            vertexes_under_x_axis = vertexes_under_x_axis + 1
            displaced_vertexes_underx = realloc_to_varray(displaced_vertexes_underx, vertexes_under_x_axis-1, free_vertexes[i])
            

    cdef size_t j
    cdef size_t k
    cdef size_t l
    cdef size_t p
    cdef double tmp_mag1
    cdef double tmp_mag2
    cdef double tmp_dot_product1
    cdef double tmp_dot_product2
    cdef Vertex tmp_placeholder1
    cdef Vertex tmp_placeholder2
    cdef Vertex tmp_vertex1
    cdef Vertex x_unit = create_vertex(1, 0, 0)


    
    for j in range(vertexes_over_x_axis):
        for k in range(vertexes_over_x_axis-j-1):
            tmp_mag1 = (magnitude(displaced_vertexes_overx[k].x-center_vertex.x, displaced_vertexes_overx[k].y-center_vertex.y))
            tmp_mag2 = (magnitude(displaced_vertexes_overx[k+1].x-center_vertex.x, displaced_vertexes_overx[k+1].y-center_vertex.x))
            if tmp_mag1 == 0.:
                return
            elif tmp_mag2 == 0.:
                return
            
            tmp_placeholder1 = displaced_vertexes_overx[k]
            tmp_placeholder1.x = (displaced_vertexes_overx[k].x-center_vertex.x)/tmp_mag1
            tmp_placeholder1.y = (displaced_vertexes_overx[k].y-center_vertex.y)/tmp_mag1
            
            tmp_placeholder2 = displaced_vertexes_overx[k+1]
            tmp_placeholder2.x = (displaced_vertexes_overx[k+1].x-center_vertex.x)/tmp_mag2
            tmp_placeholder2.y = (displaced_vertexes_overx[k+1].y-center_vertex.y)/tmp_mag2
            
            tmp_dot_product1 = dot_product(x_unit, tmp_placeholder1)
            tmp_dot_product2 = dot_product(x_unit, tmp_placeholder2)
            if tmp_dot_product1 > tmp_dot_product2:
                tmp_vertex1 = displaced_vertexes_overx[k]
                displaced_vertexes_overx[k] = displaced_vertexes_overx[k+1]
                displaced_vertexes_overx[k+1] = tmp_vertex1

                
                
    for l in range(vertexes_under_x_axis):
        for p in range(vertexes_under_x_axis-l-1):
            tmp_mag1 = (magnitude(displaced_vertexes_underx[p].x, displaced_vertexes_underx[p].y))
            tmp_mag2 = (magnitude(displaced_vertexes_underx[p+1].x, displaced_vertexes_underx[p+1].y))

            
            tmp_placeholder1 = displaced_vertexes_underx[p]
            tmp_placeholder1.x = displaced_vertexes_underx[p].x/tmp_mag1
            tmp_placeholder1.y = displaced_vertexes_underx[p].y/tmp_mag1
            
            tmp_placeholder2 = displaced_vertexes_underx[p+1]
            tmp_placeholder2.x = displaced_vertexes_underx[p+1].x/tmp_mag2
            tmp_placeholder2.y = displaced_vertexes_underx[p+1].y/tmp_mag2
            
            tmp_dot_product1 = dot_product(x_unit, tmp_placeholder1)
            tmp_dot_product2 = dot_product(x_unit, tmp_placeholder2)
            if tmp_dot_product1 > tmp_dot_product2:
                tmp_vertex1 = displaced_vertexes_underx[p]
                displaced_vertexes_underx[p] = displaced_vertexes_underx[p+1]
                displaced_vertexes_underx[p+1] = tmp_vertex1
 
    cdef size_t m
    cdef size_t n
    for m in range(vertexes_over_x_axis):
        free_vertexes[m] = displaced_vertexes_overx[m]
    

    for n in range(vertexes_over_x_axis, size):
        free_vertexes[n] = displaced_vertexes_underx[n-vertexes_over_x_axis]
        
 
    free(displaced_vertexes_overx)
    free(displaced_vertexes_underx)
#end function


"""WORK IN PROGRESS"""
"""
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Face* triangulate_vertices(Vertex* original_array, int size) nogil:
"""

"""unit test for angular sorting"""
#start the array of Zero-Zero Vertices
#!! THIS ARRAY IS MALLOCED BUT NOT FREED BY DEFAULT !! THE SIZE HAS TO BE KEPT TRACK OF
cdef int varr_size = 5
cdef Vertex* vertex_array = alloc_varr(varr_size, 0)

vertex_array[0].x = 2
vertex_array[0].y = -1
vertex_array[1].x = -0.5
vertex_array[1].y = 2
vertex_array[2].x = 3
vertex_array[2].y = -1
vertex_array[3].x = -0.5
vertex_array[3].y = -0.8
vertex_array[4].x = 5
vertex_array[4].y = 5

cdef Vertex relative_origin = create_vertex(0, 0, 17) #vid_nr to something random. not really important for this test
cdef Vertex xx_unit = create_vertex(1, 0, -1)
cdef Vertex temp_ver
cdef double mag




#test unit for bubble sort by angle on vertex array
sort_by_angle_clockwise(vertex_array, relative_origin, varr_size)





cdef Face* test_mesh = <Face*>malloc(varr_size * sizeof(Face))
#insert the last permutation to avoid % algebra
test_mesh[varr_size-1] = create_face(relative_origin, vertex_array[varr_size-1], vertex_array[0], varr_size-1)
for i in range(varr_size-1):
    test_mesh[i] = create_face(xx_unit, vertex_array[i], vertex_array[i+1], i)
    

print("printing the mesh by elements:\n")
for i in range(varr_size):
    print("face: ",i,"\n", test_mesh[i])
    

free(vertex_array)
free(test_mesh)



printing the mesh by elements:

face:  0 
 {'ver1': {'x': 1.0, 'y': 0.0, 'vid_nr': -1}, 'ver2': {'x': -0.5, 'y': 2.0, 'vid_nr': 1}, 'ver3': {'x': 5.0, 'y': 5.0, 'vid_nr': 4}, 'fid_nr': 0}
face:  1 
 {'ver1': {'x': 1.0, 'y': 0.0, 'vid_nr': -1}, 'ver2': {'x': 5.0, 'y': 5.0, 'vid_nr': 4}, 'ver3': {'x': -0.5, 'y': -0.8, 'vid_nr': 3}, 'fid_nr': 1}
face:  2 
 {'ver1': {'x': 1.0, 'y': 0.0, 'vid_nr': -1}, 'ver2': {'x': -0.5, 'y': -0.8, 'vid_nr': 3}, 'ver3': {'x': 2.0, 'y': -1.0, 'vid_nr': 0}, 'fid_nr': 2}
face:  3 
 {'ver1': {'x': 1.0, 'y': 0.0, 'vid_nr': -1}, 'ver2': {'x': 2.0, 'y': -1.0, 'vid_nr': 0}, 'ver3': {'x': 3.0, 'y': -1.0, 'vid_nr': 2}, 'fid_nr': 3}
face:  4 
 {'ver1': {'x': 0.0, 'y': 0.0, 'vid_nr': 17}, 'ver2': {'x': 3.0, 'y': -1.0, 'vid_nr': 2}, 'ver3': {'x': -0.5, 'y': 2.0, 'vid_nr': 1}, 'fid_nr': 4}


In [ ]:
##general algorithm


mesh = []
available_vertexes = []
mesh.append(super_triangle)

for vertex in vertex_list:
    for face in mesh:
        if vertex in circumcircle(face, vertex):
            face.fid_nr = -1
            face.v1 -> available_vertexes
            face.v2 -> available_vertexes
            face.v3 -> available_vertexes
    create_face(vertex, all elements in available_vertex in order) -> mesh
for face in mesh:
    if face.fid_nr = -1:
        delete face